In this notebook we combine the concepts from previous notebook. We pre-process tweets using our preprocessing pipeline, build embeddings and then classify them using a logistic regression model.

In [ ]:
!wget https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch8/O5_smtd_preprocessing.py -O "smtd_preprocessing.py"
!wget https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch8/Data/sts_gold_tweet.csv -O "sts_gold_tweet.csv"
!pip3 install demoji

--2021-11-07 10:15:56--  https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch8/O5_smtd_preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17420 (17K) [text/plain]
Saving to: ‘smtd_preprocessing.py’

smtd_preprocessing. 100%[===================>]  17.01K  --.-KB/s    in 0s      

2021-11-07 10:15:56 (36.9 MB/s) - ‘smtd_preprocessing.py’ saved [17420/17420]

--2021-11-07 10:15:56--  https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch8/Data/sts_gold_tweet.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, 

In [ ]:
#Making the necessary imports
import os
import sys

preprocessing_path = "smtd_preprocessing.py"
sys.path.append(os.path.abspath(preprocessing_path))

import smtd_preprocessing

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from string import punctuation

import pandas as pd
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()


#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Reading and Preprocessing
Let's read the dataset and pre-process them using the pre-processing pipeline.

In [ ]:
datapath = "" #Path to repo
df = pd.read_csv(datapath+"sts_gold_tweet.csv",
                 error_bad_lines=False,delimiter=";")
df = df.dropna(how='any')
df.drop(columns=['id'], inplace=True)
display(df.head())

#pre-process tweets using our package
df['tweet'] = df['tweet'].apply(lambda x: smtd_preprocessing.process_TweetText(x))
df['tweet_tokens'] = df['tweet'].apply(lambda x: tweet_tokenizer.tokenize(x))
df['tweet_no_stopwords'] = df['tweet_tokens'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
tweets_processed = df['tweet_tokens'].values
tweets_cat = df['polarity'].values

display(df.head())
print("Number of tweets and categories")
print(len(tweets_processed), len(tweets_cat))
print("\nExamle of polarity, processed tweet, processed tweet without stopwords")
print(tweets_cat[0],',',tweets_processed[0],',',df['tweet_no_stopwords'].values[0])

,polarity,tweet
0,0,the angel is going to miss the athlete this we...
1,0,It looks as though Shaq is getting traded to C...
2,0,@clarianne APRIL 9TH ISN'T COMING SOON ENOUGH
3,0,drinking a McDonalds coffee and not understand...
4,0,So dissapointed Taylor Swift doesnt have a Twi...


,polarity,tweet,tweet_tokens,tweet_no_stopwords
0,0,the angel is going to miss the athlete this we...,"[the, angel, is, going, to, miss, the, athlete...","[angel, going, miss, athlete, weekend]"
1,0,it looks as though shaq is getting traded to c...,"[it, looks, as, though, shaq, is, getting, tra...","[looks, though, shaq, getting, traded, clevela..."
2,0,constantnonbrandmention constantdate isn't com...,"[constantnonbrandmention, constantdate, isn't,...","[constantnonbrandmention, constantdate, coming..."
3,0,drinking a mcdonalds coffee and not understand...,"[drinking, a, mcdonalds, coffee, and, not, und...","[drinking, mcdonalds, coffee, understanding, s..."
4,0,so dissapointed taylor swift doesnt have a twi...,"[so, dissapointed, taylor, swift, doesnt, have...","[dissapointed, taylor, swift, doesnt, twitter]"


Number of tweets and categories
2034 2034

Examle of polarity, processed tweet, processed tweet without stopwords
0 , ['the', 'angel', 'is', 'going', 'to', 'miss', 'the', 'athlete', 'this', 'weekend'] , ['angel', 'going', 'miss', 'athlete', 'weekend']


## Train your own Embedding

In [ ]:
#CBOW
import time
start = time.time()
w2v_model = Word2Vec(tweets_processed,min_count=5, sg=0)
end = time.time()

print("CBOW Model Training Complete.\nTime taken for training is:{:.5f} sec ".format((end-start)))

CBOW Model Training Complete.
Time taken for training is:0.32860 sec 


In [ ]:
#Create document vectors by averaging word vectors.
def embedding_feats(list_of_lists):
    DIMENSION = 100
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        feats.append(feat_for_this/count_for_this if count_for_this > 0 else feat_for_this)        
    return feats

train_vectors = embedding_feats(df['tweet_no_stopwords'].values)
print(len(train_vectors))

2034


In [ ]:
#Take any classifier (LogisticRegression here)
classifier = LogisticRegression(random_state=2020)
train_data, test_data, train_cats, test_cats = train_test_split(train_vectors, 
                                                                df['polarity'].values)

classifier.fit(train_data, train_cats)
print("Accuracy: ", classifier.score(test_data, test_cats))
preds = classifier.predict(test_data)
print(classification_report(test_cats, preds))

Accuracy:  0.6758349705304518
              precision    recall  f1-score   support

           0       0.68      1.00      0.81       344
           4       0.00      0.00      0.00       165

    accuracy                           0.68       509
   macro avg       0.34      0.50      0.40       509
weighted avg       0.46      0.68      0.55       509

